## Querying and Organizing Data

In [48]:
import requests
import time
import os
import logging
import sys

import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import requests

In [49]:
# Set up logging (Jupyter sets up it's own so we have to add ours instead of using a basicConfig)
log = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
log.addHandler(fhandler)
log.setLevel(logging.CRITICAL)

In [50]:
# Function for performing a GET request using requests library with retries
# Sets a 5 second timeout by default
def get_request(url, parameters=None, timeout=5):
    try:
        response = requests.get(url=url, params=parameters, timeout=timeout)
    except SSLError as s:
        log.error('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        log.warn('\rRetrying.' + ' '*10)
        
        # Recusively try again
        return get_request(url, parameters)
    
    if response:
        log.debug('Got response {0}'.format(response.status_code))
        return response
    else:
        # Response is none usually means too many requests. Wait and try again 
        log.warn('No response, waiting 10 seconds...')
        time.sleep(10)
        log.warn('Retrying.')
        return get_request(url, parameters)

In [51]:
# Queries the games for a given username. To find your username, check your profile under General -> Custom URL
def get_steam_xml(username):
    if os.path.exists("steam_games.xml"):
        log.info('Steam XML is cached')
        with open("steam_games.xml", "r", encoding="utf-8") as games_file:
            contents = games_file.read()
    else:
        log.info('Steam XML needs query')
        xml_url = 'http://steamcommunity.com/id/{0}/games?tab=all&xml=1'.format(username)
        xml_contents = get_request(xml_url, timeout=5)
        with open("steam_games.xml", "w", encoding="utf-8") as games_file:
            games_file.write(xml_contents.text)

        contents = xml_contents.text

    return contents

In [52]:
# Reads the games XML returned from get_steam_xml() and outputs a pandas dataframe
def get_game_infos(username):
    steam_xml = get_steam_xml(username)
    tree = ET.ElementTree(ET.fromstring(steam_xml))
    root = tree.getroot()

    if root.find('error') is not None:
        log.error(root.find('error').text)
        raise Exception("Root not found")

    game_infos = []
    
    for game in root.iter('game'):
        app_id = game.find('appID').text
        name = game.find('name').text
        
        propertyOrDefault = lambda name, default: (game.find(name).text) if (game.find(name) is not None) else default

        # Rest of these are optional
        logo_link = propertyOrDefault('logo', '')
        store_link = propertyOrDefault('storeLink', '')
        hours_last_2_weeks = float(propertyOrDefault('hoursLast2Weeks', 0))
        hours_on_record = float(propertyOrDefault('hoursOnRecord', 0))
        stats_link = propertyOrDefault('statsLink', '')
        global_stats_link = propertyOrDefault('globalStatsLink', '')
        
        game_infos.append((app_id, name, logo_link, store_link, hours_last_2_weeks, hours_on_record,
                           stats_link, global_stats_link))

    df = pd.DataFrame.from_records(game_infos,
                                   columns=['AppId', 'Name', 'LogoLink', 'StoreLink', 'HoursLast2Weeks',
                                            'HoursOnRecord', 'StatsLink', 'GlobalStatsLink'])
    df.set_index('AppId', inplace = True)

    return df

In [53]:
# Get data from SteamSpy for each game
# The structure of game_infos must be at least two columns named 'AppId' and 'Name', with AppId being the index.
# Setting 'in_place' to true will modify the input game_infos preserving any other existing columns.
# Otherwise, they are discarded.
# pull_first_n allows limiting the number of queries to the first N found.
def get_steamspy_data(cache_file, game_infos, pull_first_n = None, in_place = False):
    cache = pd.DataFrame()
    if os.path.exists(cache_file):
        cache = pd.read_csv(cache_file, index_col = False)
    
    steamspy_data = []
    pulled = 0
    
    for index, row in game_infos.iterrows():
        name = row['Name']
        appid = index

        cache_found = False
        if cache.empty == False:
            log.info("Found {0} in cache".format(name))
            cache_row = cache.loc[cache['Name'] == name]
            if cache_row.empty == False:
                steamspy_data.append((appid,
                                      name,
                                      int(cache_row["Positive"]),
                                      int(cache_row["Negative"]),
                                      int(cache_row["TotalRatings"]),
                                      float(cache_row["RatingsRatio"]),
                                      int(cache_row["UserScore"]),
                                      int(cache_row["AvgForever"]),
                                      int(cache_row["Avg2Weeks"]),
                                      int(cache_row["MedForever"]),
                                      int(cache_row["Med2Weeks"])))

                cache_found = True

        if cache_found == False:
            log.info("Request {0} from SteamSpy".format(name))
                
            url = "http://steamspy.com/api.php"
            parameters = {"request": "appdetails", "appid": appid}
            json_data = get_request(url, parameters = parameters).json()
            game_info = pd.DataFrame.from_dict(json_data, orient = 'index')
            for game in game_info:
                positive = json_data["positive"]
                negative = json_data["negative"]
                total_ratings = positive + negative
                if total_ratings > 0:
                    ratings_ratio = (positive / total_ratings) * 100
                else:
                    ratings_ratio = 0
                user_score = json_data["userscore"]
                avg_forever = json_data["average_forever"]
                avg_2weeks = json_data["average_2weeks"]
                med_forever = json_data["median_forever"]
                med_2weeks = json_data["median_2weeks"]            

                log.debug("Finished request for {0}".format(name))
                steamspy_data.append((appid,
                                      name,
                                      positive,
                                      negative,
                                      total_ratings,
                                      ratings_ratio,
                                      user_score,
                                      avg_forever,
                                      avg_2weeks,
                                      med_forever,
                                      med_2weeks))                
                                          
                # Per documentation, don't make more than 1 request per second
                time.sleep(2)
        
        pulled = pulled + 1
        if pull_first_n is not None:
            log.debug("Pulled {0} of {1}".format(pulled, pull_first_n))
            if pulled == pull_first_n:
                break

    if in_place == False:
        log.info("Returning new data frame")
        merged_game_infos = pd.DataFrame.from_records(steamspy_data,
                          columns = ['AppId',
                                     'Name',
                                     'Positive',
                                     'Negative',
                                     'TotalRatings',
                                     'RatingsRatio',
                                     'UserScore',
                                     'AvgForever',
                                     'Avg2Weeks',
                                     'MedForever',
                                     'Med2Weeks'
                                    ])
        merged_game_infos.to_csv(cache_file)
        return merged_game_infos
    else:
        log.info("Appending columns to existing data frame")
        # Add the new columns to the existing game_infos
        game_infos['Positive'] = [i[2] for i in steamspy_data]
        game_infos['Negative'] = [i[3] for i in steamspy_data]
        game_infos['TotalRatings'] = [i[4] for i in steamspy_data]
        game_infos['RatingsRatio'] = [i[5] for i in steamspy_data]
        game_infos['UserScore'] = [i[6] for i in steamspy_data]
        game_infos['AvgForever'] = [i[7] for i in steamspy_data]
        game_infos['Avg2Weeks'] = [i[8] for i in steamspy_data]
        game_infos['MedForever'] = [i[9] for i in steamspy_data]
        game_infos['Med2Weeks'] = [i[10] for i in steamspy_data]
        
        game_infos.to_csv(cache_file)
        return game_infos

In [54]:
# Add a Bayesian average to better rank the games
def p_calculate_bayesian_average(item_num_ratings, item_ratio_ratings,
                system_avg_num_ratings, system_ratio_ratings):
    b_avg = (((item_num_ratings) / (item_num_ratings + system_avg_num_ratings)) * item_ratio_ratings) + (((system_avg_num_ratings) /  (item_num_ratings + system_avg_num_ratings)) * system_ratio_ratings)
    return b_avg

def add_bayesian_average_to_gamespy_dataframe(steamspy_df):
    # Calculate an overall average for the system
    system_ratings_avg = steamspy_df["RatingsRatio"].mean()
    system_num_ratings_avg = steamspy_df["TotalRatings"].mean()
        
    # Calculate Bayesian average
    b_averages = list(steamspy_df.apply(lambda row:
        p_calculate_bayesian_average(row["TotalRatings"], row["RatingsRatio"],
                    system_num_ratings_avg, system_ratings_avg), axis=1))

    # Add the new averages to the data frame
    steamspy_df['BayesianAverage'] = b_averages

In [55]:
# Note, your 'Game details' must be set to 'Public' for this to work.
# This is done in your profile -> Edit Profile -> Privacy Settings -> Game details
# To find your username, check your profile under General -> Custom URL
username = ''
if username == '':
    if os.path.exists("steam_id.dat"):
        log.info('Reading steam ID from file')
        with open("steam_id.dat", "r", encoding="utf-8") as id_file:
            username = id_file.read()
            game_infos = get_game_infos(username)
    else:
        raise Exception("Missing steam id")
        log.critical('Need steam user ID')
else:
    # Note: get_game_info does not check if username matches in case the file is already cached.
    game_infos = get_game_infos(username)

In [58]:
# Decorate our steam library info with ranking info from SteamSpy
steamspy_data = get_steamspy_data("steam_spy_cache.csv", game_infos, in_place = , pull_first_n = 5)
add_bayesian_average_to_gamespy_dataframe(steamspy_data)

ValueError: Length of values (5) does not match length of index (218)

In [59]:
# Do any filtering or re-arranging you want to here
steamspy_data = steamspy_data.sort_values(by=['BayesianAverage'], ascending=False)

# DLCs have no ratings, drop them from the list
steamspy_data.drop(steamspy_data[steamspy_data["RatingsRatio"] == 0].index, inplace = True)

# Write to file for easy access
steamspy_data.to_csv("bayesian.csv")

## Plotting

In [46]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, Label, LabelSet
from bokeh.palettes import Turbo256 as palette
from bokeh.transform import linear_cmap

In [47]:
# Plot most played games (ignore non played games)
colName = "HoursOnRecord"
output_file("MostPlayed.html")

most_played_games = pd.DataFrame.copy(steamspy_data)
most_played_games.drop(most_played_games[most_played_games[colName] == 0].index, inplace = True)

# Take just top 30
most_played_games = most_played_games.nlargest(30, colName)
most_played_games = most_played_games.sort_values(by=[colName], ascending=True)

tooltips = [
    ('Game', '@Name'),
    ('Hours Played', '@HoursOnRecord'),
    ('Rating', '@BayesianAverage')
]

select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']

color_mapper = linear_cmap(field_name = colName,
                           palette=palette,
                           low=min(most_played_games[colName]),
                           high=max(most_played_games[colName]))

# Weird issue here where the text in LabelSet must be a string or it won't work, so decorate the data with strings
most_played_games["{0}Text".format(colName)] = most_played_games[colName].apply(lambda x: str(x))
data_source = ColumnDataSource(most_played_games)

p = figure(y_range = most_played_games["Name"],
           plot_width = 2000, plot_height = 1250, title = "Most Played Games of All Time", tools = select_tools,
           x_range = (0, max(most_played_games[colName] + 20)),)

p.title.text_font_size = '32pt'
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis[0].axis_label = 'Hours Played'

p.hbar(y = "Name", left = 0, right = colName, height = 0.5, source = data_source, color = color_mapper)
p.add_tools(HoverTool(tooltips=tooltips))


labels = LabelSet(x = colName, y = "Name", level = 'annotation', text_color = 'black',
                  x_offset = 5, y_offset = -6, text = "{0}Text".format(colName), source = data_source, render_mode = 'canvas')

p.add_layout(labels)

show(p)

KeyError: 'HoursOnRecord'

In [ ]:
# Plot most played games in last 2 weeks
colName = "HoursLast2Weeks"
output_file("MostPlayedLast2Weeks.html")

most_played_games_2w = pd.DataFrame.copy(steamspy_data)
most_played_games_2w.drop(most_played_games_2w[most_played_games_2w[colName] == 0].index, inplace = True)
most_played_games_2w = most_played_games_2w.sort_values(by=[colName], ascending=False)


# Weird issue here where the text in LabelSet must be a string or it won't work, so decorate the data with strings
most_played_games_2w["{0}Text".format(colName)] = most_played_games_2w[colName].apply(lambda x: "{:.2f}".format(x))
data_source = ColumnDataSource(most_played_games_2w)

p = figure(x_range = most_played_games_2w["Name"], y_range = (0, max(most_played_games_2w[colName] + 20)),
           width = 2000, title = "Most Played Games in Last 2 Weeks")

p.vbar(x = "Name", top = colName, source = data_source, width = 0.5)
p.xaxis.major_label_orientation = "vertical"
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.yaxis[0].axis_label = 'Hours Played'

labels = LabelSet(x = "Name", y = colName, level = 'annotation', text_color = 'black',
                  x_offset = -15, y_offset = 10, text = "{0}Text".format(colName), source = data_source, render_mode = 'canvas')

p.add_layout(labels)

show(p)

In [ ]:
# Plot most played games versus their rating
colName = "HoursOnRecord"
output_file("MostPlayedVsRating.html")

most_played_games = pd.DataFrame.copy(steamspy_data)
most_played_games = most_played_games.sort_values(by=[colName], ascending=False)

tooltips = [
    ('Game', '@Name'),
    ('Hours Played', '@HoursOnRecord'),
    ('Rating', '@BayesianAverage')
]

select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']

color_mapper = linear_cmap(field_name = "HoursOnRecord",
                           palette=palette,
                           low=min(most_played_games[colName]),
                           high=max(most_played_games[colName]))

select_tools = ['tap', 'reset', 'box_zoom']

data_source = ColumnDataSource(most_played_games)
p = figure(plot_height = 1000,
           plot_width = 2000,
           title = "Most played vs ranking",
           tools = select_tools)

p.circle(x = colName, y = "BayesianAverage",
         color = color_mapper,
         source = data_source, size = 10)

p.xaxis[0].axis_label = 'Hours Played'
p.yaxis[0].axis_label = 'Positive vs Negative Rating % Adjusted Using a Bayesian average'

p.add_tools(HoverTool(tooltips=tooltips))
show(p)

In [ ]:
# Plot best ranked unplayed games
colName = "RatingsRatio"
output_file("UnplayedPlainRating.html")

tooltips = [
    ('Game', '@Name'),
    ('Rating', '@RatingsRatio')
]

select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']

best_unplayed_games = pd.DataFrame.copy(steamspy_data)
best_unplayed_games.drop(best_unplayed_games[best_unplayed_games["HoursOnRecord"] != 0].index, inplace = True)

# Remove DLC and tools (0% and 100% rated)
best_unplayed_games.drop(best_unplayed_games[best_unplayed_games[colName] == 0].index, inplace = True)
best_unplayed_games.drop(best_unplayed_games[best_unplayed_games[colName] == 100].index, inplace = True)

# Take just top 30
best_unplayed_games = best_unplayed_games.nlargest(50, colName)
best_unplayed_games = best_unplayed_games.sort_values(by=[colName], ascending=True)

color_mapper = linear_cmap(field_name = colName,
                           palette=palette,
                           low=min(best_unplayed_games[colName]),
                           high=max(best_unplayed_games[colName]))

# Weird issue here where the text in LabelSet must be a string or it won't work, so decorate the data with strings
best_unplayed_games["{0}Text".format(colName)] = best_unplayed_games[colName].apply(lambda x: "{:.2f}".format(x))

data_source = ColumnDataSource(best_unplayed_games)

min_range = int(min(best_unplayed_games[colName]) - 1)
max_range = int(max(best_unplayed_games[colName]) + 2)
p = figure(y_range = best_unplayed_games["Name"], x_range = (min_range, max_range),
           plot_width = 2000, plot_height = 1250, title = "Best unplayed games", tools = select_tools)

p.title.text_font_size = '32pt'
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis[0].axis_label = 'Positive vs. Negative Ratings %'

p.hbar(y = "Name", left = 0, right = colName, height = 0.5, source = data_source, color = color_mapper)

labels = LabelSet(x = colName, y = "Name", level = 'annotation', text_color = 'black',
                  x_offset = 5, y_offset = -6, text = "{0}Text".format(colName), source = data_source, render_mode = 'canvas')

p.add_layout(labels)

p.add_tools(HoverTool(tooltips=tooltips))
show(p)

In [ ]:
# Plot best ranked unplayed games
colName = "BayesianAverage"
output_file("UnplayedBayesian.html")

tooltips = [
    ('Game', '@Name'),
    ('Rating', '@BayesianAverage')
]

select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']

best_unplayed_games = pd.DataFrame.copy(steamspy_data)
best_unplayed_games.drop(best_unplayed_games[best_unplayed_games["HoursOnRecord"] != 0].index, inplace = True)

# Remove DLC and tools (0% and 100% rated)
best_unplayed_games.drop(best_unplayed_games[best_unplayed_games[colName] == 0].index, inplace = True)
best_unplayed_games.drop(best_unplayed_games[best_unplayed_games[colName] == 100].index, inplace = True)

# Take just top 30
best_unplayed_games = best_unplayed_games.nlargest(50, colName)
best_unplayed_games = best_unplayed_games.sort_values(by=[colName], ascending=True)

color_mapper = linear_cmap(field_name = colName,
                           palette=palette,
                           low=min(best_unplayed_games[colName]),
                           high=max(best_unplayed_games[colName]))

# Weird issue here where the text in LabelSet must be a string or it won't work, so decorate the data with strings
best_unplayed_games["{0}Text".format(colName)] = best_unplayed_games[colName].apply(lambda x: "{:.2f}".format(x))

data_source = ColumnDataSource(best_unplayed_games)

min_range = int(min(best_unplayed_games[colName]) - 1)
max_range = int(max(best_unplayed_games[colName]) + 2)
p = figure(y_range = best_unplayed_games["Name"], x_range = (min_range, max_range),
           plot_width = 2000, plot_height = 1250, title = "Best unplayed games", tools = select_tools)

p.title.text_font_size = '32pt'
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis[0].axis_label = 'Positive vs. Negative Ratings % Adjusted Using a Bayesian Average'

p.hbar(y = "Name", left = 0, right = colName, height = 0.5, source = data_source, color = color_mapper)

labels = LabelSet(x = colName, y = "Name", level = 'annotation', text_color = 'black',
                  x_offset = 5, y_offset = -6, text = "{0}Text".format(colName), source = data_source, render_mode = 'canvas')

p.add_layout(labels)

p.add_tools(HoverTool(tooltips=tooltips))
show(p)